In [43]:
import csv
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException

# ChromeDriver 路徑
driver_path = r"C:\chromedriver-win64\chromedriver.exe"
options = Options()
options.add_argument('--incognito')

# 建立 WebDriver
service = Service(driver_path)
driver = webdriver.Chrome(service=service, options=options)

# 開啟網站
url = "https://www.louisacoffee.co/visit"
driver.get(url)

# 等待頁面加載
time.sleep(3)

# 點擊查詢按鈕
try:
    search_button = driver.find_element(By.XPATH, '//button[@type="submit"]')
    search_button.click()
    print("查詢按鈕已點擊，開始加載門市資料...")
except Exception as e:
    print(f"查詢按鈕點擊失敗：{e}")
    driver.quit()

# 等待頁面加載
time.sleep(3)

# 模擬滾動直到加載完所有門市
scroll_pause_time = 2  # 每次滾動之後的等待時間
scroll_height = driver.execute_script("return document.body.scrollHeight")  # 取得當前頁面的高度
while True:
    # 滾動到底部
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(scroll_pause_time)

    # 計算新的頁面高度，並與之前的比較
    new_scroll_height = driver.execute_script("return document.body.scrollHeight")
    if new_scroll_height == scroll_height:  # 如果頁面高度沒有變化，則結束滾動
        break
    scroll_height = new_scroll_height

print("已加載所有門市資料，開始爬取...")

# 欲爬取的門市型態條件
conditions = [
    "親子友善空間", "手沖體驗門市", "座位30席以上", "輕食早午餐", "手做蛋糕",
    "貝果", "免費Wifi", "會議型空間租借", "高鐵TGO兌換"
]

# 定義 CSV 標題
csv_headers = ["門市名稱", "電話", "地址", "營業時間"] + conditions

# 初始化資料存儲
store_data = []

# 爬取資料
stores = driver.find_elements(By.CSS_SELECTOR, ".row .col-md-6.store_info")  # 找到所有門市區塊

for store in stores:
    # 提取基本資料
    store_name = store.find_element(By.TAG_NAME, "h4").text.strip()
    phone = store.find_element(By.XPATH, ".//p[contains(text(), '電話')]").text.replace("電話/", "").strip()
    address = store.find_element(By.XPATH, ".//p[contains(text(), '地址')]").text.replace("地址/", "").strip()

    # 處理營業時間，防止找不到元素的錯誤
    try:
        business_hours = store.find_element(By.XPATH, ".//p[contains(text(), '營業時間')]").text.replace("營業時間/", "").strip()
    except NoSuchElementException:
        business_hours = "未知"  # 若找不到，設為「未知」

    # 提取門市型態條件
    condition_status = {}
    for condition in conditions:
        condition_status[condition] = "無"  # 預設為 "無"

    # 爬取附加條件並檢查是否符合指定條件
    for condition in conditions:
        try:
            # 檢查是否有符合條件的元素
            condition_element = store.find_element(By.XPATH, f".//p[contains(text(), '{condition}')]/i[@class='fa fa-check-square-o']")
            if condition_element:
                condition_status[condition] = "有"  # 若有條件，設為 "有"
        except NoSuchElementException:
            # 若找不到該條件，則跳過
            pass

    # 建立資料列
    row = [store_name, phone, address, business_hours] + [condition_status[cond] for cond in conditions]
    store_data.append(row)

# 將資料寫入 CSV 檔案
csv_file = "louisa_stores.csv"

with open(csv_file, mode="w", newline="", encoding="utf-8-sig") as file:
    writer = csv.writer(file)
    writer.writerow(csv_headers)  # 寫入標題
    writer.writerows(store_data)  # 寫入資料

print(f"資料已成功儲存到 {csv_file}！")

print(f"已成功抓取所有門市資料，共計 {len(store_data)} 間，保存於 {csv_file}！")

# 關閉瀏覽器
driver.quit()


查詢按鈕已點擊，開始加載門市資料...
已加載所有門市資料，開始爬取...
資料已成功儲存到 louisa_stores.csv！
已成功抓取所有門市資料，共計 534 間，保存於 louisa_stores.csv！


In [33]:
import csv
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By

# ChromeDriver 路徑
driver_path = r"C:\chromedriver-win64\chromedriver.exe"
options = webdriver.ChromeOptions()
options.add_argument('--incognito')

# 建立 WebDriver
service = Service(driver_path)
driver = webdriver.Chrome(service=service, options=options)

# 開啟網站
url = "https://www.louisacoffee.co/visit"  # 替換為實際目標網址
driver.get(url)

# 等待網站加載
driver.implicitly_wait(10)

# 欲爬取的門市型態條件
conditions = [
    "親子友善空間", "手沖體驗門市", "座位30席以上", "輕食早午餐", "手做蛋糕",
    "貝果", "免費Wifi", "會議型空間租借", "高鐵TGO兌換"
]

# 定義 CSV 標題
csv_headers = ["門市名稱", "電話", "地址", "營業時間"] + conditions

# 爬取資料
store_data = []
stores = driver.find_elements(By.CSS_SELECTOR, ".row .col-md-6.store_info")  # 找到所有門市區塊

for store in stores:
    # 提取基本資料
    store_name = store.find_element(By.TAG_NAME, "h4").text.strip()
    phone = store.find_element(By.XPATH, ".//p[contains(text(), '電話')]").text.replace("電話/", "").strip()
    address = store.find_element(By.XPATH, ".//p[contains(text(), '地址')]").text.replace("地址/", "").strip()
    business_hours = store.find_element(By.XPATH, ".//p[contains(text(), '營業時間')]").text.replace("營業時間/", "").strip()

    # 提取門市型態條件
    condition_status = {}
    for condition in conditions:
        condition_status[condition] = "無"  # 預設為 "無"

    # 爬取附加條件並檢查是否符合指定條件
    extra_conditions_elements = store.find_elements(By.XPATH, ".//p[contains(@class, 'fa-check-square-o')]")
    for condition_element in extra_conditions_elements:
        condition_text = condition_element.text.strip()
        if condition_text in conditions:
            condition_status[condition_text] = "有"

    # 建立資料列
    row = [store_name, phone, address, business_hours] + [condition_status[cond] for cond in conditions]
    store_data.append(row)

# 將資料寫入 CSV 檔案
csv_file = "louisa_stores.csv"

with open(csv_file, mode="w", newline="", encoding="utf-8-sig") as file:
    writer = csv.writer(file)
    writer.writerow(csv_headers)  # 寫入標題
    writer.writerows(store_data)  # 寫入資料

print(f"資料已成功儲存到 {csv_file}！")

# 關閉瀏覽器
driver.quit()


查詢按鈕已點擊，開始加載門市資料...
已加載所有門市資料，開始爬取...
已成功抓取所有門市資料，共計 534 間，保存於 louisa_stores_all_with_conditions.csv！
